In [1]:
import os

In [2]:
%pwd

'c:\\Work\\Tasks\\Repo\\uc_api\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Work\\Tasks\\Repo\\uc_api'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    # Common Configurations
    root_dir: Path
    dataset_path: Path
    tokenizer_name: str

    # Common Parameters
    data_split_type: list 

    # Parameters
    feature_column_name: str 
    label_column_name: str 
    sentiment_class_dict: dict 
    batch_size: int
    max_length: int

In [6]:
from src.uc_api.constants import *
from src.uc_api.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        # Reading yaml files for config and params
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        # Common configurations from yaml file
        config = self.config.data_transformation

        # Common Parameters from yaml file
        params_common = self.params.CommonArguments

        # Parameters picked from use-case specific parameters
        params = self.params.SentimentArguments        

        create_directories([config.root_dir])

        # Populating the data class 
        data_transformation_config = DataTransformationConfig(
            # Common Configurations
            root_dir=config.root_dir,
            dataset_path=config.dataset_path,
            tokenizer_name=config.tokenizer_name,

            # Common Parameters
            data_split_type=params_common.data_split_type,

            # Use-case specific params -- Sentiment            
            feature_column_name=params.feature_column_name,
            label_column_name=params.label_column_name,
            sentiment_class_dict = params.sentiment_class_dict,
            batch_size=params.batch_size,
            max_length=params.max_length,
        )

        return data_transformation_config

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader


from transformers import BertTokenizer
from datasets import load_from_disk 
from src.uc_api.logging import logger

[2024-06-10 10:07:39: INFO:config: PyTorch version 2.3.1 available.]


In [9]:
class SentimentDataset(Dataset):
    def __init__(self, text, sentimentRating, tokenizer, max_length):
        self.text = text
        self.sentimentRating = sentimentRating
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        text = str(self.text[item])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return {
            "text": text,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "sentimentRating": torch.tensor(
                self.sentimentRating[item], dtype=torch.long
            ),
        }

In [10]:
class SentimentDataTransformation:

    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_and_save_to_file(self):
        # loading the dataloder for transformation
        dataset = load_from_disk(self.config.dataset_path)
        # print(data_set)
        
        for dataset_type in self.config.data_split_type:
            ds = SentimentDataset(

                # feature
                text=dataset[dataset_type][self.config.feature_column_name],

                # Label in negative: 0, neutral: 1, positive: 2
                sentimentRating=[self.config.sentiment_class_dict[element] for element in dataset[dataset_type][self.config.label_column_name]],
                tokenizer=BertTokenizer.from_pretrained(
                    self.config.tokenizer_name,
                ),
                # 
                max_length=self.config.max_length,
            )
            # Saving the dataloader based on the data split type
            torch.save( 
                DataLoader(
                    ds,
                    batch_size=self.config.batch_size,
                ),
                os.path.join(self.config.root_dir, f"{dataset_type}_dataloader.pt"),
            )
            logger.info(
                f"""Successfully saved {dataset_type} dataloader to {self.config.root_dir}"""
            )

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = SentimentDataTransformation(config=data_transformation_config)
    data_transformation.transform_and_save_to_file()
except Exception as e:
    logger.info("Error creating dataloader")
    raise e

[2024-06-10 10:07:40: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-06-10 10:07:40: INFO:common: yaml file: params.yaml loaded successfully]
[2024-06-10 10:07:40: INFO:common: created directory at: artifacts]
[2024-06-10 10:07:40: INFO:common: created directory at: artifacts/data_transformation]
[2024-06-10 10:07:40: INFO:3235233465: Successfully saved train dataloader to artifacts/data_transformation]
[2024-06-10 10:07:41: INFO:3235233465: Successfully saved test dataloader to artifacts/data_transformation]
[2024-06-10 10:07:41: INFO:3235233465: Successfully saved validation dataloader to artifacts/data_transformation]
